In [8]:
# autoreload
%load_ext autoreload
%autoreload 2

In [59]:
import numpy as np
from eugene import datasets
from eugene import dataload as dl
from eugene import preprocess as pp

In [2]:
sdata = datasets.random1000()

In [3]:
pp.ohe_seqs_sdata(sdata)

One-hot encoding sequences:   0%|          | 0/1000 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added


In [55]:
sdataset = dl.SequenceDataset(
    seqs=sdata.ohe_seqs,
    targets=sdata["label_0"].values,
    transforms=dl.ReverseComplement(p=0.5)
)

In [56]:
sdataloader = sdataset.to_dataloader(
    batch_size=32,
    shuffle=True,
    num_workers=0,
)

In [57]:
next(iter(sdataloader))["seq"].shape

torch.Size([32, 4, 100])

In [48]:
from eugene import models

In [70]:
model = models.DeepBind(input_len=100, output_dim=1)

In [71]:
model(next(iter(sdataloader))["seq"]).shape

torch.Size([32, 1])

In [72]:
pp.train_test_split_sdata(sdata, train_key="train_val")

In [82]:
train_idx = np.where(sdata.seqs_annot["train_val"] == True)[0]
val_idx = np.where(sdata.seqs_annot["train_val"] == False)[0]

train_dataset = dl.SequenceDataset(
    seqs=sdata.ohe_seqs[train_idx],
    targets=sdata["label_0"].values[train_idx],
    transforms=dl.ReverseComplement(p=0.5)
)

val_dataset = dl.SequenceDataset(
    seqs=sdata.ohe_seqs[val_idx],
    targets=sdata["label_0"].values[val_idx],
)   

In [83]:
train_dataloader = train_dataset.to_dataloader(
    batch_size=32,
    shuffle=True,
    num_workers=0,
)

val_dataloader = val_dataset.to_dataloader(
    batch_size=32,
    shuffle=False,
    num_workers=0,
)

In [76]:
model(next(iter(train_dataloader))["seq"]).shape, model(next(iter(val_dataloader))["seq"]).shape

(torch.Size([32, 1]), torch.Size([32, 1]))

In [77]:
from eugene import train

In [3]:
import torch

In [4]:
torch.cuda.is_available()

False

In [88]:
train.fit(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    gpus=1,
    epochs=10
)

[rank: 0] Global seed set to 13
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type        | Params
---------------------------------------------
0 | train_metric | R2Score     | 0     
1 | val_metric   | R2Score     | 0     
2 | test_metric  | R2Score     | 0     
3 | conv1d_tower | Conv1DTower | 1.0 K 
4 | max_pool     | MaxPool1d   | 0     
5 | avg_pool     | AvgPool1d   | 0     
6 | dense_block  | DenseBlock  | 1.1 K 
---------------------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [7]:
model

NameError: name 'model' is not defined